---

### 🎓 **Professor**: Apostolos Filippas

### 📘 **Class**: AI Engineering

### 📋 **Topic**: Lexical Search & BM25

🚫 **Note**: You are not allowed to share the contents of this notebook with anyone outside this class without written permission by the professor.

---

## Welcome!

In this lecture, we'll explore **lexical search** — the foundation of many search engines like Elasticsearch and Vespa, but also an amazing tool for context engineering -- and an amazing tool in the hands of AI Agents. 

Don't worry if this seems like a lot — we'll take it step by step!

In [ ]:
import pandas as pd # you know what pandas is -- it's a data analysis library
import numpy as np # a numerical computing library
from collections import Counter # a dict that counts the number of times each element appears in a list
from typing import Callable # a type hint for functions
from string import punctuation # a string of all the punctuation characters
import warnings 
warnings.filterwarnings("ignore") 
pd.set_option('display.max_colwidth', None)

In [ ]:
# example with counter 
Counter([1,2,3,4,5,6,7,8,9,10,10,10,2])

In [ ]:
punctuation

---

# 1. Introduction to Lexical Search

## 1.1 What is Lexical Search?

**Lexical search** finds documents by matching the exact words (tokens) in your query against words in your documents.

> 📚 **TERM: Lexical Search**  
> A search method that matches documents based on exact string/token matches. "Lexical" refers to words or vocabulary — the search operates on the literal text.

Think of it like using `Ctrl+F` in a document, but smarter:
- It can match across many documents
- It ranks results by relevance
- You control exactly what counts as a "match"

## 1.2 Lexical vs. Embedding-Based Search (next week)

| Aspect | Lexical Search | Embedding Search |
|--------|---------------|------------------|
| **How it works** | Exact token matching | Semantic similarity |
| **Control** | High — you decide what matches | Low — model decides |
| **Speed** | Very fast (inverted index) | Slower (vector math) |
| **Best for** | Precise queries, keywords, IDs | Fuzzy queries, synonyms |

**Key insight**: Embedding search is a **sledgehammer** — powerful but imprecise. Lexical search is a **scalpel** — precise and controllable.

## 1.3 The Inverted Index

> 📚 **TERM: Inverted Index**  
> A data structure that maps each unique term to the list of documents containing it. This enables fast lookups: instead of scanning all documents, you look up which documents contain your search term.

```python
Regular index:  Doc1 → ["hello", "world"]
                Doc2 → ["hello", "there"]

Inverted index: "hello" → [Doc1, Doc2]
                "world" → [Doc1]
                "there" → [Doc2]
```

---

# 2. Tokenization

Before we can search, we need to break text into searchable units called **tokens**.

## 2.1 Index-Time Tokenization

Let's start with some sample data — a chat transcript:

In [ ]:
# Sample chat data
chat_transcript = [
    "Hi this is Apostolos, I'd like to complain about the weather",
    "Apostolos, this is Tom, support for Earth's Climate, how can we help?",
    "Tom, can I speak to your manager?",
    "Hi, this is Sue, Tom's boss. What can I do for you?",
    "I'd like to complain about the ski conditions in Greece",
    "Oh apostolos thats terrible, lets see what we can do."
]

msgs = pd.DataFrame({
    "name": ["Apostolos", "Tom", "Apostolos", "Sue", "Apostolos", "Sue"],
    "msg": chat_transcript
})
msgs

### Simple Whitespace Tokenization

The simplest tokenizer just splits on whitespace:

In [ ]:
def whitespace_tokenize(text: str) -> list[str]:
    """Split text on whitespace."""
    return text.split()

In [ ]:
whitespace_tokenize("Mary had a little lamb")

### The Problem: "Apostolos," != "apostolos"

Let's see what happens when we tokenize our messages:

In [ ]:
# Look at how "Apostolos" appears in different messages
print("Message 0:", whitespace_tokenize(chat_transcript[0]))
print("Message 5:", whitespace_tokenize(chat_transcript[5]))

Notice the problem:
- `"Apostolos,"` (with comma) ≠ `"apostolos"` (lowercase)
- These are **different strings**, so they won't match!

**Takeaway**: Lexical search is about _extremely precise control of string matching_. YOU decide what to accept as equivalent words.

### Better Tokenization

Let's improve our tokenizer:
1. Lowercase everything
2. Remove punctuation
3. Split on whitespace

In [ ]:
def better_tokenize(text: str) -> list[str]:
    """Tokenize with lowercase and punctuation removal."""
    lowercased = text.lower()
    without_punctuation = lowercased.translate(str.maketrans('', '', punctuation))
    return without_punctuation.split()


In [ ]:
# Test it
print("Before:", whitespace_tokenize("Apostolos, that weirdo?"))
print("After: ", better_tokenize("Apostolos, that weirdo?"))

Now `"Apostolos,"` and `"apostolos"` both become `"apostolos"`!

### Building an Inverted Index from Scratch

Let's build our own inverted index using just Python dictionaries:

In [ ]:
def build_index(docs: list[str], tokenizer: Callable[[str], list[str]]) -> tuple[dict[str, dict[int, int]], list[int]]:
    """
    Build an inverted index from a list of documents.
    
    Returns:
        index: dict mapping term -> {doc_id: term_count}
        doc_lengths: list of document lengths (in tokens)
    """
    index = {}  # term -> {doc_id: count}
    doc_lengths = []
    
    for doc_id, doc in enumerate(docs):
        tokens = tokenizer(doc)
        doc_lengths.append(len(tokens))
        
        # Count term frequencies in this document
        term_counts = Counter(tokens)
        
        for term, count in term_counts.items():
            if term not in index:
                index[term] = {}
            index[term][doc_id] = count
    
    return index, doc_lengths

In [ ]:
# Build the index
index, doc_lengths = build_index(msgs['msg'].tolist(), better_tokenize)

In [ ]:
index

In [ ]:
doc_lengths

In [ ]:
# Let's look at the index entry for "apostolos"
print("Documents containing 'apostolos':")
print(index.get('apostolos', {}))

## 2.2 Query-Time Tokenization

Now let's search! We need to tokenize the query the same way we tokenized the documents.

### Simple Search Function

In [ ]:
def search(query: str, index: dict[str, dict[int, int]], num_docs: int, tokenizer: Callable[[str], list[str]]) -> set[int]:
    """
    Search the index for documents matching the query.
    Returns set of matching document IDs.
    """
    query_tokens = tokenizer(query)
    matching_docs = set()
    
    for token in query_tokens:
        if token in index:
            matching_docs.update(index[token].keys())
    
    return matching_docs

# Search for "apostolos"
results = search("apostolos", index, len(msgs), better_tokenize)
print(f"Documents matching 'apostolos': {results}")
msgs.iloc[list(results)]

### OR Queries vs AND Queries

What if we search for multiple terms like "apostolos complaint"?

- **OR query**: Match documents containing ANY of the terms
- **AND query**: Match documents containing ALL of the terms

In [ ]:
def search_or(query, index, num_docs, tokenizer):
    """OR query: match ANY term."""
    query_tokens = tokenizer(query)
    matches = np.zeros(num_docs, dtype=bool)
    
    for token in query_tokens:
        if token in index:
            for doc_id in index[token]:
                matches[doc_id] = True  # OR: set to True if any term matches
    
    return matches

def search_and(query, index, num_docs, tokenizer):
    """AND query: match ALL terms."""
    query_tokens = tokenizer(query)
    matches = np.ones(num_docs, dtype=bool)  # Start with all True
    
    for token in query_tokens:
        token_matches = np.zeros(num_docs, dtype=bool)
        if token in index:
            for doc_id in index[token]:
                token_matches[doc_id] = True
        matches &= token_matches  # AND: must match all terms
    
    return matches

In [ ]:
# OR query: "apostolos" OR "complaint"
or_matches = search_or("apostolos complain", index, len(msgs), better_tokenize)
print("OR query results (apostolos OR complain):")
msgs[or_matches]

In [ ]:
# AND query: "apostolos" AND "complaint"
and_matches = search_and("apostolos complain", index, len(msgs), better_tokenize)
print("AND query results (apostolos AND complain):")
msgs[and_matches]

---

🛑 **TRY IT NOW**
1. Search for "tom manager" with OR query — how many results?
2. Search for "tom manager" with AND query — how many results?
3. What happens if you search for a word that doesn't exist?



In [ ]:
# search for "tom manager" with OR query 

In [ ]:
# search for "tom manager" with AND query 

---

# 3. Scoring with TF*IDF

So far we've only checked if documents match. But how do we **rank** them by relevance?

## 3.1 Term Frequency (TF)

> 📚 **TERM: Term Frequency (TF)**  
> The number of times a term appears in a document. More occurrences = more relevant (to a point).

Let's build helper functions to get term statistics:

In [ ]:
def get_tf(term, doc_id, index):
    """Get term frequency for a term in a document."""
    if term in index and doc_id in index[term]:
        return index[term][doc_id]
    return 0

def get_df(term, index):
    """Get document frequency for a term (how many docs contain it)."""
    if term in index:
        return len(index[term])
    return 0

# Test it
print(f"TF of 'apostolos' in doc 0: {get_tf('apostolos', 0, index)}")
print(f"TF of 'apostolos' in doc 5: {get_tf('apostolos', 5, index)}")
print(f"DF of 'apostolos': {get_df('apostolos', index)} documents")

## 3.2 Document Frequency (DF) and IDF

> 📚 **TERM: Inverse Document Frequency (IDF)**  
> A measure of how rare/specific a term is. IDF = 1 / DF (approximately). Rare terms have high IDF; common terms have low IDF.

**Why IDF matters**: If a term appears in every document, it's not useful for distinguishing between them. Rare terms are more informative!

In [ ]:
# Compare common vs rare terms
print("Common terms (appear in many docs):")
print(f"  'this' appears in {get_df('this', index)} docs")
print(f"  'can' appears in {get_df('can', index)} docs")

print("\nRare terms (appear in few docs):")
print(f"  'virginia' appears in {get_df('virginia', index)} docs")
print(f"  'manager' appears in {get_df('manager', index)} docs")

## 3.3 TF*IDF Scoring

> 📚 **TERM: TF*IDF**  
> A scoring formula that multiplies Term Frequency by Inverse Document Frequency. Documents score high when they contain the search term frequently AND the term is rare across documents.

Let's implement TF*IDF scoring:

In [ ]:
def score_tfidf(query, index, num_docs, doc_lengths, tokenizer):
    """
    Score documents using TF*IDF.
    Returns array of scores for each document.
    """
    query_tokens = tokenizer(query)
    scores = np.zeros(num_docs)
    
    for token in query_tokens:
        df = get_df(token, index)
        if df == 0:
            continue
        
        idf = 1.0 / df  # Simple IDF
        
        for doc_id in range(num_docs):
            tf = get_tf(token, doc_id, index)
            scores[doc_id] += tf * idf
    
    return scores

In [ ]:
# Score documents for "apostolos complain"
scores = score_tfidf("apostolos complain", index, len(msgs), doc_lengths, better_tokenize)

msgs_with_scores = msgs.copy()
msgs_with_scores['score'] = scores
msgs_with_scores.sort_values('score', ascending=False)

### Why Rare Terms Score Higher

Let's see the individual term contributions:

In [ ]:
# Break down scoring by term
query = "apostolos complain"
query_tokens = better_tokenize(query)

for token in query_tokens:
    df = get_df(token, index)
    idf = 1.0 / df if df > 0 else 0
    print(f"Term '{token}': DF={df}, IDF={idf:.3f}")
    
    # Show which docs have this term
    if token in index:
        for doc_id, tf in index[token].items():
            print(f"  Doc {doc_id}: TF={tf}, score contribution={tf * idf:.3f}")

---

# 4. Multi-Field Search

Real documents often have multiple fields (title, body, tags). Let's search across multiple fields!

## 4.1 Indexing Multiple Fields

In [ ]:
# Build separate indices for message and name fields
msg_index, msg_lengths = build_index(msgs['msg'].tolist(), better_tokenize)
name_index, name_lengths = build_index(msgs['name'].tolist(), better_tokenize)

print(f"Message index has {len(msg_index)} unique terms")
print(f"Name index has {len(name_index)} unique terms")

## 4.2 Stemming for Normalization

> 📚 **TERM: Stemming**  
> Reducing words to their root form. For example: "complaint", "complaining", "complained" all become "complain". This helps match different forms of the same word.

Let's add a simple stemmer to our tokenizer:

In [ ]:
# Simple suffix-stripping stemmer (for demonstration)
def simple_stem(word):
    """Very basic stemmer - removes common suffixes."""
    suffixes = ['ing', 'ed', 'er', 'est', 's', 'ly']
    for suffix in suffixes:
        if word.endswith(suffix) and len(word) > len(suffix) + 2:
            return word[:-len(suffix)]
    return word

def stemming_tokenize(text):
    """Tokenize with stemming."""
    tokens = better_tokenize(text)
    return [simple_stem(token) for token in tokens]

# Test stemming
print("Without stemming:", better_tokenize("I have complaints about complaining"))
print("With stemming:   ", stemming_tokenize("I have complaints about complaining"))

In [ ]:
# Rebuild indices with stemming
msg_index_stem, msg_lengths = build_index(msgs['msg'].tolist(), stemming_tokenize)
name_index_stem, name_lengths = build_index(msgs['name'].tolist(), stemming_tokenize)

# Now "complaint" and "complain" map to the same token
print("Docs containing 'complain' (stemmed):")
print(msg_index_stem.get('complain', {}))

## 4.3 The Problem with Naive Score Summing

If we just add scores from multiple fields, we run into problems:

In [ ]:
def score_multifield_naive(query, indices, num_docs, tokenizer):
    """Naive multi-field scoring: just sum scores from all fields."""
    query_tokens = tokenizer(query)
    scores = np.zeros(num_docs)
    
    for token in query_tokens:
        for field_name, index in indices.items():
            df = get_df(token, index)
            if df == 0:
                continue
            idf = 1.0 / df
            
            for doc_id in range(num_docs):
                tf = get_tf(token, doc_id, index)
                scores[doc_id] += tf * idf
    
    return scores

indices = {'msg': msg_index_stem, 'name': name_index_stem}
scores = score_multifield_naive("apostolos complain", indices, len(msgs), stemming_tokenize)

msgs_with_scores = msgs.copy()
msgs_with_scores['score'] = scores
msgs_with_scores.sort_values('score', ascending=False)

**Problem**: A document matching "apostolos" twice (once in name, once in message) scores higher than a document matching both "apostolos" AND "complain"!

## 4.4 Dismax: Term-Centric Search

> 📚 **TERM: Dismax (Disjunction Maximum)**  
> For each query term, take the MAXIMUM score across all fields instead of summing. This prevents the same term from being counted multiple times.

This is called **term-centric** search:

In [ ]:
def score_multifield_dismax(query, indices, num_docs, tokenizer):
    """Dismax multi-field scoring: max score per term across fields."""
    query_tokens = tokenizer(query)
    scores = np.zeros(num_docs)
    
    for token in query_tokens:
        # For this term, find the best score across all fields
        term_scores = np.zeros(num_docs)
        
        for field_name, index in indices.items():
            df = get_df(token, index)
            if df == 0:
                continue
            idf = 1.0 / df
            
            for doc_id in range(num_docs):
                tf = get_tf(token, doc_id, index)
                field_score = tf * idf
                term_scores[doc_id] = max(term_scores[doc_id], field_score)
        
        scores += term_scores  # Add the best score for this term
    
    return scores

scores = score_multifield_dismax("apostolos complain", indices, len(msgs), stemming_tokenize)

msgs_with_scores = msgs.copy()
msgs_with_scores['score'] = scores
msgs_with_scores.sort_values('score', ascending=False)

Now documents matching BOTH terms rank higher!

---

# 5. From TF*IDF to BM25

Our simple TF*IDF has two problems that BM25 solves:

## 5.1 Problems with Naive TF*IDF

### Problem 1: Term Frequency Saturation

With raw TF, a document mentioning "apostolos" 10 times scores 10x higher than one mentioning it once. But does it really make the document 10x more relevant?

This is called **keyword stuffing** — repeating words doesn't make a document more relevant after a point.

In [ ]:
# Demo: keyword stuffing problem
stuffed_docs = [
    "Apostolos has a complaint",
    "Apostolos Apostolos Apostolos Apostolos Apostolos Apostolos Apostolos Apostolos Apostolos Apostolos loves repetition"
]

stuffed_index, stuffed_lengths = build_index(stuffed_docs, better_tokenize)

# With raw TF, the stuffed document wins
for doc_id, doc in enumerate(stuffed_docs):
    tf = get_tf('apostolos', doc_id, stuffed_index)
    print(f"Doc {doc_id}: TF={tf}, Doc: '{doc[:50]}...'")

### Problem 2: Field Length Bias

A term appearing once in a 5-word tweet is more significant than appearing once in a 5000-word article. The tweet is clearly "about" that term; the article just mentions it in passing.

## 5.2 Fixing Saturation

We can use logarithmic saturation to dampen the effect of high term frequencies:

In [ ]:
# Log saturation
raw_tfs = [1, 2, 5, 10, 20, 50, 100]
saturated = [np.log(tf + 1) for tf in raw_tfs]

print("Raw TF vs Log-Saturated TF:")
for raw, sat in zip(raw_tfs, saturated):
    print(f"  TF={raw:3d} → {sat:.2f}")

## 5.3 Fixing Length Bias

We normalize by document length:

In [ ]:
# Length normalization example
tf = 1  # Same term frequency
short_doc_len = 10
long_doc_len = 1000

print(f"TF=1 in {short_doc_len}-word doc: normalized = {tf / short_doc_len:.4f}")
print(f"TF=1 in {long_doc_len}-word doc: normalized = {tf / long_doc_len:.4f}")

## 5.4 BM25: The Formula

> 📚 **TERM: BM25 (Best Match 25)**  
> A refined scoring formula that addresses TF saturation and length bias. "25" refers to it being the 25th iteration of the Best Match formula.

The BM25 formula has two key parameters:
- **k1** (typically 1.2): Controls how fast TF saturates
- **b** (typically 0.75): Controls length normalization (0 = none, 1 = full)

```
BM25 score = IDF × (TF × (k1 + 1)) / (TF + k1 × (1 - b + b × docLen/avgDocLen))
```

In [ ]:
def bm25_idf(df, num_docs):
    """BM25 IDF formula."""
    return np.log((num_docs - df + 0.5) / (df + 0.5) + 1)

def bm25_tf(tf, doc_len, avg_doc_len, k1=1.2, b=0.75):
    """BM25 TF normalization."""
    return (tf * (k1 + 1)) / (tf + k1 * (1 - b + b * doc_len / avg_doc_len))

def score_bm25(query, index, num_docs, doc_lengths, tokenizer, k1=1.2, b=0.75):
    """Score documents using BM25."""
    query_tokens = tokenizer(query)
    scores = np.zeros(num_docs)
    avg_doc_len = np.mean(doc_lengths)
    
    for token in query_tokens:
        df = get_df(token, index)
        if df == 0:
            continue
        
        idf = bm25_idf(df, num_docs)
        
        for doc_id in range(num_docs):
            tf = get_tf(token, doc_id, index)
            if tf > 0:
                tf_norm = bm25_tf(tf, doc_lengths[doc_id], avg_doc_len, k1, b)
                scores[doc_id] += idf * tf_norm
    
    return scores

In [ ]:
# Compare TF*IDF vs BM25 on keyword-stuffed documents
stuffed_index, stuffed_lengths = build_index(stuffed_docs, better_tokenize)

print("Keyword-stuffed documents:")
for i, doc in enumerate(stuffed_docs):
    print(f"  Doc {i}: '{doc}'")

print("\nScoring for query 'apostolos':")

tfidf_scores = score_tfidf("apostolos", stuffed_index, len(stuffed_docs), stuffed_lengths, better_tokenize)
bm25_scores = score_bm25("apostolos", stuffed_index, len(stuffed_docs), stuffed_lengths, better_tokenize)

print(f"  TF*IDF: {tfidf_scores}")
print(f"  BM25:   {bm25_scores}")

BM25 reduces the gap between the stuffed and non-stuffed document!

---

# 6. BM25F: Multi-Field BM25

## 6.1 The Field Blending Problem

When we run BM25 on each field separately and then combine scores, we have a problem: IDF is calculated **per field**.

A term might be rare in the "title" field but common in the "body" field. Taking the max gives the title field too much weight.

In [ ]:
# Demo the problem
docs_with_topics = [
    {"msg": "Hi this is Apostolos, I have a complaint about the weather", "topics": "weather complaint"},
    {"msg": "Apostolos, we hear you. What's the issue apostolos?", "topics": "earth climate"},
    {"msg": "Tom, can I speak to your manager?", "topics": "escalation"},
    {"msg": "I have complaints about skiing", "topics": "skiing complaint"},
    {"msg": "Thanks you guys are great", "topics": "gratitude"},
    {"msg": "That's very sweet", "topics": "apostolos"}  # "apostolos" is rare in topics!
]

msgs_topics = pd.DataFrame(docs_with_topics)

msg_idx, msg_lens = build_index(msgs_topics['msg'].tolist(), better_tokenize)
topic_idx, topic_lens = build_index(msgs_topics['topics'].tolist(), better_tokenize)

print(f"'apostolos' DF in msg field: {get_df('apostolos', msg_idx)}")
print(f"'apostolos' DF in topics field: {get_df('apostolos', topic_idx)}")

The term "apostolos" appears in only 1 document's topics field, making it seem very rare/important there, even though it's common overall.

## 6.2 BM25F Solution

> 📚 **TERM: BM25F (BM25 for structured documents with Fields)**  
> An extension of BM25 that blends term frequencies across fields BEFORE applying IDF. This ensures term rarity is measured across the entire corpus, not per-field.

Instead of:
```
score = BM25(field1) + BM25(field2)  # IDF calculated per field
```

We do:
```
combined_tf = TF(field1) + TF(field2)
combined_df = max(DF(field1), DF(field2))  # Or sum
score = BM25_formula(combined_tf, combined_df)
```

In [ ]:
def score_bm25f(query, indices, doc_lengths_dict, num_docs, tokenizer, k1=1.2, b=0.75):
    """
    BM25F scoring: blend TF across fields before applying IDF.
    
    Args:
        indices: dict of {field_name: index}
        doc_lengths_dict: dict of {field_name: [lengths]}
    """
    query_tokens = tokenizer(query)
    scores = np.zeros(num_docs)
    
    # Calculate average doc length across all fields
    total_lengths = np.zeros(num_docs)
    for field_lengths in doc_lengths_dict.values():
        total_lengths += np.array(field_lengths)
    avg_doc_len = np.mean(total_lengths)
    
    for token in query_tokens:
        # Calculate combined DF (max across fields)
        combined_df = 0
        for index in indices.values():
            combined_df = max(combined_df, get_df(token, index))
        
        if combined_df == 0:
            continue
        
        # Use combined DF for IDF
        idf = bm25_idf(combined_df, num_docs)
        
        for doc_id in range(num_docs):
            # Combine TF across fields (with length normalization per field)
            combined_impact = 0
            
            for field_name, index in indices.items():
                tf = get_tf(token, doc_id, index)
                if tf > 0:
                    doc_len = doc_lengths_dict[field_name][doc_id]
                    avg_field_len = np.mean(doc_lengths_dict[field_name])
                    # Length-normalized impact
                    impact = tf / (1 - b + b * doc_len / avg_field_len)
                    combined_impact += impact
            
            if combined_impact > 0:
                # Apply TF saturation to combined impact
                saturated = combined_impact / (combined_impact + k1)
                scores[doc_id] += idf * saturated
    
    return scores

In [ ]:
# Compare naive multi-field vs BM25F
indices = {'msg': msg_idx, 'topics': topic_idx}
lengths = {'msg': msg_lens, 'topics': topic_lens}

naive_scores = score_multifield_dismax("apostolos complaint", indices, len(msgs_topics), better_tokenize)
bm25f_scores = score_bm25f("apostolos complaint", indices, lengths, len(msgs_topics), better_tokenize)

comparison = msgs_topics.copy()
comparison['naive'] = naive_scores
comparison['bm25f'] = bm25f_scores
comparison.sort_values('bm25f', ascending=False)

BM25F gives more balanced results by measuring term rarity across the entire corpus!

---

# 🎯 Summary

## What We Covered

| Concept | What It Is | Key Formula/Approach |
|---------|-----------|---------------------|
| **Tokenization** | Breaking text into searchable terms | lowercase + punctuation removal |
| **Inverted Index** | Maps terms to documents | `term → {doc_id: count}` |
| **TF** | Term Frequency | Count of term in document |
| **IDF** | Inverse Document Frequency | `1 / (docs containing term)` |
| **TF*IDF** | Basic relevance scoring | `TF × IDF` |
| **BM25** | Improved TF*IDF | Saturation (k1) + length norm (b) |
| **BM25F** | Multi-field BM25 | Blend TF before IDF |

## ✅ Can You Do These?

- [ ] Build a tokenizer that normalizes text
- [ ] Create an inverted index from scratch
- [ ] Implement OR and AND queries
- [ ] Calculate TF, DF, and TF*IDF scores
- [ ] Explain why rare terms score higher
- [ ] Implement BM25 with k1 and b parameters
- [ ] Use dismax for multi-field search
- [ ] Implement BM25F field blending

## 🆘 Troubleshooting

| Problem | Solution |
|---------|----------|
| No search results | Check tokenization — are query terms matching index terms? |
| Wrong ranking | Review scoring function — TF saturation and length norm help |
| Multi-field bias | Use dismax or BM25F, not naive sum |
| Keyword stuffing dominates | Use BM25's TF saturation (k1 parameter) |

## Next Class

We'll explore **embedding-based search** and learn when to combine lexical and semantic approaches!

---

## Resources

- [BM25 Explained](https://www.elastic.co/blog/practical-bm25-part-1-how-shards-affect-relevance-scoring-in-elasticsearch)
- [Elasticsearch Analyzers](https://www.elastic.co/docs/reference/text-analysis/analyzer-reference)
- [Introduction to Information Retrieval (Stanford)](https://nlp.stanford.edu/IR-book/)